In [1]:
# for article content scripting with BeautifulSoup
import requests
from bs4 import BeautifulSoup
import re

# for rendering html template with Jinja
from jinja2 import Environment, FileSystemLoader
import os

# for building and sending emails with MailChimp
import mailchimp_marketing as MailchimpMarketing
from mailchimp_marketing.api_client import ApiClientError
import json

# for viewing the email draft in webpage
import webbrowser
from datetime import date

# Define the Article Object with BeautifulSoup

In [16]:
class Article():
    
    def __init__(self, url, title=None, img_url=None, preview_text=None, tag=None, label=None, class_name='article', id_name='main'):
        
        self._url = url.replace('/gb/', '/b5/')
        self._title = title
        self._img_url = img_url
        self._preview_text = preview_text
        self._tag = tag
        self._label = label
        
        page = requests.get(self._url)
        self._soup = BeautifulSoup(page.text, 'html.parser')
        
        self._article = self._soup.find(class_=class_name)
        
        if not self._article:
            self._article = self._soup.find(id=id_name)
            
        if not self._article:
            self._article = self._soup.find('article')
        
    @property
    def url(self):
        return self._url
    
    @url.setter
    def url(self, value):
        self._url = value
        
    @property
    def title(self, class_name='title'):
        
        if not self._title:
            article_title = self._article.find(class_=class_name)
            self._title = article_title.text
            
        return self._title
    
    @title.setter
    def title(self, value):
        self._title = value
    
    @property
    def img_url(self, class_name='featured_image'):
        
        if not self._img_url:
            try:
                self._img_url = self._article.find(class_=class_name).find('a').get('href')
            except:
                self._img_url = self._article.find(class_='wp-post-image').get('src')
        return self._img_url
    
    @img_url.setter
    def img_url(self, value):
        self._img_url = value
    
    @property
    def preview_text(self, id_name='artbody'):
        
        paragraph_index = 0
        
        while not self._preview_text:
            article_first_paragraph = self._article.find(id=id_name).find_all('p')[paragraph_index].text
            article_preview_text = re.sub(r'\【[^)]*\】', '', article_first_paragraph)
            article_preview_text = re.sub(r'\n', '', article_preview_text) 
            article_preview_text = re.sub(r'\（英文大纪元[^)]*\编译）', '', article_preview_text)
            article_preview_text = re.sub(r'\（英文大紀元[^)]*\編譯）', '', article_preview_text)
            article_preview_text = re.sub(r'\（大紀元[^)]*\報導）', '', article_preview_text)
            article_preview_text = re.sub(r'\（大紀元[^)]*\報導）', '', article_preview_text)
            self._preview_text = re.sub(r'\（大纪元[^)]*\报导）', '', article_preview_text)
            paragraph_index += 1
        
        return self._preview_text
    
    @preview_text.setter
    def preview_text(self, value):
        self._preview_text = value
        
    @property
    def tag(self, id_name='breadcrumb'):
        
        if not self._tag:
            breadcrumb = self._soup.find(id=id_name)
            raw_tag = breadcrumb.findAll('a')[-1].text
            raw_tag = re.sub(r'\([^)]*\)', '', raw_tag)
            self._tag = re.sub(r'\（[^)]*\）', '', raw_tag)
        
        return self._tag
    
    @tag.setter
    def tag(self, value):
        self._tag = value
        
    @property
    def label(self):
        return self._label
    
    @label.setter
    def label(self, value):
        self._label = value

In [17]:
class Content(Article):
    
    def __init__(self, article_url_list, video_url_list):
        
        # parse data
        self.article_list = [Article(article_url) for article_url in article_url_list]
        
        self.video_list = [Article(video_url) for video_url in video_url_list]
        self.video_pair_list = [[value, self.video_list[counter+1]] for counter, value in enumerate(self.video_list) if counter%2 == 0]      

In [18]:
class Jinja_Template(Content):
    
    def __init__(self, 
                 template_folder_name = 'Jinja Templates', 
                 template_file_name = 'newsletter_as_jinja_template_base.html'):

        file_loader = FileSystemLoader('../'+template_folder_name+'/')
        self.env = Environment(loader=file_loader, extensions=['jinja2.ext.do'])

        # render the base template
        self.template = self.env.get_template(template_file_name)
        
    def render(self, source_contect, 
               folder_name='Exported HTML', 
               rendered_newsletter_filename = 'rendered_newsletter.html'):
        
        # description: subject_line and preview_text
        self.subject_line = source_contect.article_list[0].title
        article_title_list = [article.title for article in source_contect.article_list]
        self.preview_text = ' · '.join(article_title_list[1:])
        
        self.rendered_content = self.template.render(env=self.env, 
                                                   header=source_contect.article_list[0], 
                                                   article_list=source_contect.article_list, 
                                                   video_pair_list=source_contect.video_pair_list)
        
        self.folder_path = '../' + folder_name + '/'
        
        if not os.path.exists(self.folder_path):
            os.makedirs(self.folder_path)
        
        self.newsletter_path = self.folder_path + rendered_newsletter_filename
        
        with open(self.newsletter_path, "w") as fh:
            fh.write(self.rendered_content)
            
        print('newsletter rendered at: '+ self.newsletter_path)
        
    def preview(self):

        webbrowser.open('file://' + os.path.realpath(self.newsletter_path))

In [19]:
class MailChimp_Campaign(Jinja_Template):

    def __init__(self, newsletter, mailchimp_config,
                 # sender information
                 from_name='大紀元英倫生活網',
                 reply_to='accounts_uk@epochtimes.com'):

        self.subject_line = newsletter.subject_line
        self.preview_text = newsletter.preview_text
        self.rendered_content = newsletter.rendered_content
        self.mailchimp_config = mailchimp_config

        # sender information
        self.from_name = from_name
        self.reply_to = reply_to
        
        # campaign name
        today = date.today().strftime("%d %B %Y")
        self.campaign_title = 'Campaign_Created_through_API' + today

    def connect_to_MailChimp(self):

        self.client = MailchimpMarketing.Client()
        self.client.set_config({
            "api_key": self.mailchimp_config["API_KEY"],
            "server": self.mailchimp_config["API_KEY"].split('-')[-1]
        })

        response_ping = self.client.ping.get()

        if response_ping['health_status'] == "Everything's Chimpy!":
            print("MailChimp account connected!")

    def update_MailChimp_template(self):

        response = self.client.templates.update_template(
            self.mailchimp_config['template_id'], {"name": "Template_Created_through_API", "html": self.rendered_content})
        print("Template updated at:", response['date_edited'])

    def specify_recipients(self):

        if self.mailchimp_config['segment_id']:
            self.recipients = {'segment_opts':
                               {'saved_segment_id':
                                   self.mailchimp_config['segment_id']},
                               'list_id': self.mailchimp_config['list_id']}
        else:
            self.recipients = {'list_id': self.mailchimp_config['list_id']}

        print('recipients specified.')

    def create(self):

        self.connect_to_MailChimp()
        self.update_MailChimp_template()
        self.specify_recipients()

        self.created_campaign = self.client.campaigns.create({"type": "regular",
                                                              'recipients': self.recipients,
                                                              "settings":
                                                              {"title": self.campaign_title,
                                                               "subject_line": self.subject_line,
                                                               "preview_text": self.preview_text,
                                                               "from_name": self.from_name,
                                                               "reply_to": self.reply_to,
                                                               "template_id": self.mailchimp_config['template_id'],
                                                               "auto_footer": False}})
        print('Campaign created successfully!')

    def send(self):

        campaign_id = self.created_campaign['id']
        response_of_campaigns_send = self.client.campaigns.send(campaign_id)

        return response_of_campaigns_send

# Processing Part

In [20]:
article_url_list = ['https://www.epochtimes.com/b5/21/3/24/n12832713.htm',
                    'https://www.epochtimes.com/gb/21/3/28/n12840909.htm',
                    'https://www.epochtimes.com/b5/21/3/27/n12839120.htm',
                    'https://www.epochtimes.com/b5/21/3/25/n12835770.htm']

video_url_list = ['https://www.ntdtv.com/b5/2021/03/28/a103083378.html',
                  'https://www.epochtimes.com/gb/21/3/24/n12833595.htm',
                  'https://www.epochtimes.com/gb/21/3/26/n12838788.htm',
                  'https://www.epochtimes.com/gb/21/3/27/n12839283.htm']

In [21]:
source_content = Content(article_url_list, video_url_list)

In [22]:
for article in source_content.article_list:
    print(article.title)
    print(article.tag)
    print(article.label)
    print(article.preview_text)
    print()

組圖：英國封鎖一周年 民眾反思並哀悼逝者
社會新聞
None
去年3月23日，英國首相約翰遜下令第一次封鎖。當時，染疫死亡人數不到1千，封鎖屆滿一周年時，累積死亡人數已經超過12.6萬人，是歐洲染疫死亡人數最多的國家。

【網海拾貝】英國制裁誰？中共又制裁誰？
自由廣場
None
在中共為了報復英國而於3月26日制裁了9位英國人和4個英國機構後，英相約翰遜在推特上表達了對這些英國人的支持；而英國外相拉布則表示，英國制裁的是侵犯人權者，而中共制裁的是批評中共的人，這非常能說明問題。

英國網絡安全負責人：必須警惕中共科技野心
國際要聞
None
英國網絡安全中心負責人週五（3月26日）表示，英國必須對中共在科技領域的野心「保持清醒」（Clear-Eyed），並警惕中共帶來的威脅。

英國南部城市騷亂 警方展開大規模調查
社會新聞
None
3月21日晚間，英格蘭西南城市Bristol發生騷亂。一個警察局被騷亂者圍攻，21名警察受傷，其中兩人重傷，兩輛警車被毀。目前警方正在對此事進行調查。



In [23]:
# source_content.article_list[0].title = '疫情导致去年英国经济缩水创300年记录'

In [24]:
for article in source_content.video_list:
    try:
        print(article.title)
        print(article.tag)
        print(article.preview_text)
        print(article.img_url)
        print()
    except:
        pass

【嚴真點評&外交部大實話】影評《阿拉斯加2+2》
嚴真點評
【探索時分】史上最帥戰機：F14雄貓戰機
軍事天地
大家好，今天我要介紹我心目中最帥的戰機F14 TOMCAT，雄貓。
https://i.epochtimes.com/assets/uploads/2021/03/6c51370ff327afb317c5e22fbfbcf925-600x400.jpg

【橫河直播】H&M事件效應 世界吃這一套？
每日新聞
今天是3月26日，星期五。橫河老師現場直播。
https://i.epochtimes.com/assets/uploads/2021/03/0326-1200-800-600x400.jpg

【拍案驚奇】軍機擾台 習福建督戰？李克強訪德企
新聞拍案驚奇
大家好，歡迎收看《新聞拍案驚奇》，我是大宇。
https://i.epochtimes.com/assets/uploads/2021/03/b4f9d30c5198bf66426d3bd9477370bb-600x400.jpg



In [28]:
source_content.video_list[0].img_url = 'https://i.ntdtv.com/assets/uploads/2021/03/maxresdefault-82-800x450.jpg'

In [29]:
newsletter = Jinja_Template()

In [30]:
newsletter.render(source_content)

newsletter rendered at: ../Exported HTML/rendered_newsletter.html


In [31]:
newsletter.preview()

# Connect to MailChimp

## Test Campaign

In [32]:
mailchimp_config_test = {
    "API_KEY": "deb802371b503afc57fc1044df6f7f57-us14",
    "template_id": 193010,
    "segment_id": 108502, # recipients
    "list_id": '2f7d774c42' # members list
}

### Create MailChimp campaign

In [33]:
test_campaign = MailChimp_Campaign(newsletter, mailchimp_config_test)

In [34]:
test_campaign.create()

MailChimp account connected!
Template updated at: 2021-03-28T21:47:41+00:00
recipients specified.
Campaign created successfully!


In [35]:
test_campaign.send()

<Response [204]>

## Real Campaign

In [36]:
mailchimp_config_real = {
    "API_KEY": "ed90e1ec15d234be03769c355b0ca96f-us8",
    "template_id": 476378,
    "segment_id": None, # recipients
    "list_id": '6d74ea3bf4' # members list
}

In [37]:
real_campaign = MailChimp_Campaign(newsletter, mailchimp_config_real)

In [38]:
real_campaign.create()

MailChimp account connected!
Template updated at: 2021-03-28T21:47:53+00:00
recipients specified.
Campaign created successfully!


In [39]:
real_campaign.send()

<Response [204]>

## Drafts

In [200]:
# define article tag
tag_list = ['英国新闻', '英国新闻', '英国新闻', '英国新闻']

for index in range(len(tag_list)):
    article_list[index].tag = tag_list[index]
    
# article_list[0].img_url = 'https://i.epochtimes.com/assets/uploads/2020/12/2012112056442124.jpg'